In [1]:
%pwd

'/home/mohamed-stifi/Desktop/pfa-s4/notebooks'

In [2]:
import os
os.chdir('..')

%pwd

'/home/mohamed-stifi/Desktop/pfa-s4'

In [3]:
from text_classifier.components.model_trainer import TextDataset, ModelTrainer
from text_classifier.config.configuration import ConfigurationManager

/home/mohamed-stifi/Desktop/pfa-s4


/home/mohamed-stifi/Desktop/pfa-s4/pfa-venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
config = ConfigurationManager()

model_trainer_config = config.get_model_trainer_config()
model_trainer_config

[2025-05-29 18:39:24,840: INFO: common: text_classifier.utils.common: yaml file: /home/mohamed-stifi/Desktop/pfa-s4/config/config.yaml loaded successfully]
[2025-05-29 18:39:24,908: INFO: common: text_classifier.utils.common: yaml file: /home/mohamed-stifi/Desktop/pfa-s4/config/params.yaml loaded successfully]
[2025-05-29 18:39:24,912: INFO: common: text_classifier.utils.common: created directory at: artifacts]
[2025-05-29 18:39:24,914: INFO: common: text_classifier.utils.common: created directory at: artifacts/model_trainer]


ModelTrainerConfig(root_dir='artifacts/model_trainer', data_path='artifacts/data_transformation', model_name='bert-base-uncased', num_train_epochs=5, warmup_ratio=0.1, per_device_train_batch_size=16, per_device_eval_batch_size=16, weight_decay=0.01, logging_steps=10, evaluation_strategy='epoch', eval_steps=500, save_steps=500, gradient_accumulation_steps=1, learning_rate=2e-05)

In [5]:
model_trainer_config.eval_steps = 5
model_trainer_config.save_steps = 5
model_trainer_config.per_device_eval_batch_size = 8
model_trainer_config.per_device_train_batch_size = 8

model_trainer_config

ModelTrainerConfig(root_dir='artifacts/model_trainer', data_path='artifacts/data_transformation', model_name='bert-base-uncased', num_train_epochs=5, warmup_ratio=0.1, per_device_train_batch_size=8, per_device_eval_batch_size=8, weight_decay=0.01, logging_steps=10, evaluation_strategy='epoch', eval_steps=5, save_steps=5, gradient_accumulation_steps=1, learning_rate=2e-05)

In [6]:
model_trainer = ModelTrainer(model_trainer_config)

model_trainer

## Prepare data loaders

### LSTM

In [7]:
lstm_train_loader, lstm_val_loader, lstm_test_loader, lstm_vocab_size = model_trainer.prepare_data_loaders('lstm')

In [8]:
for batch in lstm_train_loader:
    print(batch.keys())
    print('input_ids: ', batch['input_ids'].shape)
    print('attention_mask: ', batch['attention_mask'].shape)
    print('labels: ', batch['labels'].shape)

    break

dict_keys(['input_ids', 'attention_mask', 'labels'])
input_ids:  torch.Size([8, 512])
attention_mask:  torch.Size([8, 512])
labels:  torch.Size([8])


In [9]:
lstm_vocab_size

3197

In [10]:
lstm = model_trainer.get_model('lstm', lstm_vocab_size)
lstm

LSTMClassifier(
  (embedding): Embedding(3197, 128, padding_idx=0)
  (lstm): LSTM(128, 256, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (classifier): Linear(in_features=512, out_features=2, bias=True)
  (criterion): CrossEntropyLoss()
)

In [11]:
test_results, model_path = model_trainer.train_model('lstm')

[2025-05-28 23:51:20,839: INFO: model_trainer: textClassifierLogger: Starting training for lstm]
[2025-05-28 23:51:21,395: INFO: setup: pytorch_lightning.utilities.rank_zero: GPU available: False, used: False]
[2025-05-28 23:51:21,400: INFO: setup: pytorch_lightning.utilities.rank_zero: TPU available: False, using: 0 TPU cores]
[2025-05-28 23:51:21,402: INFO: setup: pytorch_lightning.utilities.rank_zero: HPU available: False, using: 0 HPUs]
[2025-05-28 23:51:21,435: INFO: model_summary: pytorch_lightning.callbacks.model_summary: 
  | Name       | Type             | Params | Mode 
--------------------------------------------------------
0 | embedding  | Embedding        | 409 K  | train
1 | lstm       | LSTM             | 2.4 M  | train
2 | dropout    | Dropout          | 0      | train
3 | classifier | Linear           | 1.0 K  | train
4 | criterion  | CrossEntropyLoss | 0      | train
--------------------------------------------------------
2.8 M     Trainable params
0         Non-tra

/home/mohamed-stifi/Desktop/pfa-s4/pfa-venv/lib/python3.12/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /home/mohamed-stifi/Desktop/pfa-s4/artifacts/model_trainer/lstm exists and is not empty.


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/home/mohamed-stifi/Desktop/pfa-s4/pfa-venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


/home/mohamed-stifi/Desktop/pfa-s4/pfa-venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Epoch 3:  28%|██▊       | 5/18 [02:40<06:57,  0.03it/s, v_num=ae42, train_loss=0.668, train_acc=0.750, val_loss=0.679, val_acc=0.800]
[2025-05-29 00:21:44,347: INFO: checkpoint_connector: pytorch_lightning.utilities.rank_zero: Restoring states from the checkpoint path at /home/mohamed-stifi/Desktop/pfa-s4/artifacts/model_trainer/lstm/lstm-epoch=02-val_acc=0.80.ckpt]
[2025-05-29 00:21:44,533: INFO: checkpoint_connector: pytorch_lightning.utilities.rank_zero: Loaded model weights from the checkpoint at /home/mohamed-stifi/Desktop/pfa-s4/artifacts/model_trainer/lstm/lstm-epoch=02-val_acc=0.80.ckpt]


/home/mohamed-stifi/Desktop/pfa-s4/pfa-venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 5/5 [00:02<00:00,  1.85it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc                   0.75
         test_f1            0.7395833134651184
     test_precision         0.7976190447807312
       test_recall                 0.75
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


2025/05/29 00:22:25 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2025-04-15; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'text-classifier'}
2025/05/29 00:22:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[2025-05-29 00:22:25,886: INFO: model_trainer: textClassifierLogger: Training completed for lstm]
[2025-05-29 00:22:25,888: INFO: model_trainer: textClassifierLogger: Test results: {'test_acc': 0.75, 'test_precision': 0.7976190447807312, 'test_recall': 0.75, 'test_f1': 0.7395833134651184}]


In [7]:
model_trainer.config.num_train_epochs = 1

### bert

In [8]:
bert = model_trainer.get_model('bert')

In [9]:
for name, param in bert.named_parameters():
    if param.requires_grad:
        print(f"Trainable: {name}")


Trainable: classifier.weight
Trainable: classifier.bias


In [10]:
sum(p.numel() for p in bert.parameters() if p.requires_grad)

1538

In [11]:
test_results, model_path = model_trainer.train_model('bert')

[2025-05-29 02:09:30,676: INFO: model_trainer: textClassifierLogger: Starting training for bert]
[2025-05-29 02:09:32,073: INFO: setup: pytorch_lightning.utilities.rank_zero: GPU available: False, used: False]
[2025-05-29 02:09:32,076: INFO: setup: pytorch_lightning.utilities.rank_zero: TPU available: False, using: 0 TPU cores]
[2025-05-29 02:09:32,078: INFO: setup: pytorch_lightning.utilities.rank_zero: HPU available: False, using: 0 HPUs]
[2025-05-29 02:09:32,149: INFO: model_summary: pytorch_lightning.callbacks.model_summary: 
  | Name       | Type             | Params | Mode 
--------------------------------------------------------
0 | bert       | BertModel        | 109 M  | eval 
1 | dropout    | Dropout          | 0      | train
2 | classifier | Linear           | 1.5 K  | train
3 | criterion  | CrossEntropyLoss | 0      | train
--------------------------------------------------------
1.5 K     Trainable params
109 M     Non-trainable params
109 M     Total params
437.935   Tota

/home/mohamed-stifi/Desktop/pfa-s4/pfa-venv/lib/python3.12/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /home/mohamed-stifi/Desktop/pfa-s4/artifacts/model_trainer/bert exists and is not empty.


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/home/mohamed-stifi/Desktop/pfa-s4/pfa-venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


/home/mohamed-stifi/Desktop/pfa-s4/pfa-venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Epoch 0: 100%|██████████| 18/18 [05:53<00:00,  0.05it/s, v_num=ae87, train_loss=0.749, train_acc=0.500, val_loss=0.759, val_acc=0.050]
[2025-05-29 02:16:01,999: INFO: checkpoint_connector: pytorch_lightning.utilities.rank_zero: Restoring states from the checkpoint path at /home/mohamed-stifi/Desktop/pfa-s4/artifacts/model_trainer/bert/bert-epoch=00-val_acc=0.10.ckpt]
[2025-05-29 02:16:10,710: INFO: checkpoint_connector: pytorch_lightning.utilities.rank_zero: Loaded model weights from the checkpoint at /home/mohamed-stifi/Desktop/pfa-s4/artifacts/model_trainer/bert/bert-epoch=00-val_acc=0.10.ckpt]


/home/mohamed-stifi/Desktop/pfa-s4/pfa-venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 5/5 [00:58<00:00,  0.09it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc                   0.125
         test_f1            0.12445278465747833
     test_precision         0.12406015396118164
       test_recall                 0.125
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


2025/05/29 02:17:58 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2025-04-15; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'text-classifier'}
2025/05/29 02:17:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[2025-05-29 02:18:02,183: INFO: model_trainer: textClassifierLogger: Training completed for bert]
[2025-05-29 02:18:02,189: INFO: model_trainer: textClassifierLogger: Test results: {'test_acc': 0.125, 'test_precision': 0.12406015396118164, 'test_recall': 0.125, 'test_f1': 0.12445278465747833}]


In [12]:
test_results, model_path 

({'test_acc': 0.125,
  'test_precision': 0.12406015396118164,
  'test_recall': 0.125,
  'test_f1': 0.12445278465747833},
 'artifacts/model_trainer/bert/model.ckpt')

### roberta

In [8]:
roberta = model_trainer.get_model('roberta')

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
for name, param in roberta.named_parameters():
    if param.requires_grad:
        print(f"Trainable: {name}")


Trainable: classifier.weight
Trainable: classifier.bias


In [10]:
sum(p.numel() for p in roberta.parameters() if p.requires_grad)

1538

In [11]:
test_results, model_path = model_trainer.train_model('roberta')


[2025-05-29 02:36:45,129: INFO: model_trainer: textClassifierLogger: Starting training for roberta]


2025/05/29 02:36:45 INFO mlflow.tracking.fluent: Experiment with name 'text_classification_roberta' does not exist. Creating a new experiment.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[2025-05-29 02:36:47,570: INFO: setup: pytorch_lightning.utilities.rank_zero: GPU available: False, used: False]
[2025-05-29 02:36:47,577: INFO: setup: pytorch_lightning.utilities.rank_zero: TPU available: False, using: 0 TPU cores]
[2025-05-29 02:36:47,583: INFO: setup: pytorch_lightning.utilities.rank_zero: HPU available: False, using: 0 HPUs]
[2025-05-29 02:36:47,813: INFO: model_summary: pytorch_lightning.callbacks.model_summary: 
  | Name       | Type             | Params | Mode 
--------------------------------------------------------
0 | roberta    | RobertaModel     | 124 M  | eval 
1 | dropout    | Dropout          | 0      | train
2 | classifier | Linear           | 1.5 K  | train
3 | criterion  | CrossEntropyLoss | 0      | train
--------------------------------------------------------
1.5 K     Trainable params
124 M     Non-trainable params
124 M     Total params
498.589   Total estimated model params size (MB)
3         Modules in train mode
228       Modules in eval mode

/home/mohamed-stifi/Desktop/pfa-s4/pfa-venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


/home/mohamed-stifi/Desktop/pfa-s4/pfa-venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


...

Testing DataLoader 0: 100%|██████████| 5/5 [01:24<00:00,  0.06it/s]

/home/mohamed-stifi/Desktop/pfa-s4/pfa-venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc                    0.5
         test_f1            0.3333333432674408
     test_precision                0.25
       test_recall                  0.5
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


2025/05/29 02:44:57 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2025-04-15; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'text-classifier'}
2025/05/29 02:44:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[2025-05-29 02:45:15,016: INFO: model_trainer: textClassifierLogger: Training completed for roberta]
[2025-05-29 02:45:15,046: INFO: model_trainer: textClassifierLogger: Test results: {'test_acc': 0.5, 'test_precision': 0.25, 'test_recall': 0.5, 'test_f1': 0.3333333432674408}]


## train all

In [13]:
help(model_trainer)

Help on ModelTrainer in module text_classifier.components.model_trainer object:

class ModelTrainer(builtins.object)
 |  ModelTrainer(config: text_classifier.entity.config_entity.ModelTrainerConfig)
 |
 |  Methods defined here:
 |
 |  __init__(self, config: text_classifier.entity.config_entity.ModelTrainerConfig)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  get_model(self, model_name, vocab_size=None)
 |      Get model based on model name
 |
 |  prepare_data_loaders(self, model_name)
 |      Prepare data loaders for training
 |
 |  train_all_models(self)
 |      Train all models and compare results
 |
 |  train_model(self, model_name)
 |      Train a specific model
 |
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |
 |  __dict__
 |      dictionary for instance variables
 |
 |  __weakref__
 |      list of weak references to the object



In [ ]:
comparison_results = model_trainer.train_all_models()

[2025-05-29 16:07:07,809: INFO: model_trainer: textClassifierLogger: Starting training for lstm]
[2025-05-29 16:07:11,386: INFO: setup: pytorch_lightning.utilities.rank_zero: GPU available: False, used: False]
[2025-05-29 16:07:11,389: INFO: setup: pytorch_lightning.utilities.rank_zero: TPU available: False, using: 0 TPU cores]
[2025-05-29 16:07:11,394: INFO: setup: pytorch_lightning.utilities.rank_zero: HPU available: False, using: 0 HPUs]
[2025-05-29 16:07:11,560: INFO: model_summary: pytorch_lightning.callbacks.model_summary: 
  | Name       | Type             | Params | Mode 
--------------------------------------------------------
0 | embedding  | Embedding        | 409 K  | train
1 | lstm       | LSTM             | 2.4 M  | train
2 | dropout    | Dropout          | 0      | train
3 | classifier | Linear           | 1.0 K  | train
4 | criterion  | CrossEntropyLoss | 0      | train
--------------------------------------------------------
2.8 M     Trainable params
0         Non-tra

/home/mohamed-stifi/Desktop/pfa-s4/pfa-venv/lib/python3.12/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /home/mohamed-stifi/Desktop/pfa-s4/artifacts/model_trainer/lstm exists and is not empty.


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/mohamed-stifi/Desktop/pfa-s4/pfa-venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


/home/mohamed-stifi/Desktop/pfa-s4/pfa-venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


...

/home/mohamed-stifi/Desktop/pfa-s4/pfa-venv/lib/python3.12/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /home/mohamed-stifi/Desktop/pfa-s4/artifacts/model_trainer/bert exists and is not empty.


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/home/mohamed-stifi/Desktop/pfa-s4/pfa-venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


/home/mohamed-stifi/Desktop/pfa-s4/pfa-venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Epoch 0: 100%|██████████| 18/18 [08:11<00:00,  0.04it/s, v_num=8ee4, train_loss=0.591, train_acc=0.750, val_loss=0.657, val_acc=0.500]
[2025-05-29 16:27:33,705: INFO: checkpoint_connector: pytorch_lightning.utilities.rank_zero: Restoring states from the checkpoint path at /home/mohamed-stifi/Desktop/pfa-s4/artifacts/model_trainer/bert/bert-epoch=00-val_acc=0.50-v2.ckpt]
